# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd

In [2]:
from pymystem3 import Mystem
m = Mystem() 


In [3]:
from collections import Counter

In [4]:
df = pd.read_csv('1_borrowers.csv')

Изучим первые 10 строк датасета

In [5]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь посмотрим на последние десять строк

In [6]:
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Для полноты картины выведем общую информацию о таблице.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Предварительный просмотр общей информации о таблице позволяет увидеть в столбцах 'days_employed'('общий трудовой стаж в днях') и 'total_income'('ежемесячный доход') равное количество пропущенных значений.

Возникает гипотеза о том, что, возможно, существует какая-то связь между пропущенными значениями и теми или иными категориями клиентов. Гипотезу необходимо подтвердить или опровергнуть.

Для того, чтобы сформировалось более полное предварительное представление о особенностях необработанных данных в таблице, рассмотрим её повнимательнее, пробежимся по столбцам и, при необходимости, сопоставим их друг с другом.

In [8]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Итак, как и было отмечено выше, мы имеем одинаковое количество пропусков в столбцах с трудовым стажем и ежемесячным доходом. Теперь мы видим точное количество пропущенных значений - 2174.

In [9]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

В наименованиях столбцов таких проблем как пропуски или некорректные названия не обнаружены. Оставим им изначальные названия.

Рассмотрим уникальные значения по типу занятости

In [10]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Для наглядности пропусков значений по столбцам напишем функцию, принимающую на вход имя столбца и возвращающую количество пропусков для каждого уникального значения этого столбца:

In [11]:
def skip_value(column):
    
    values = df[column].value_counts().keys().tolist()
    counts = df[column].value_counts().tolist()
    res_temp1 = []
    
    for i in values:
        temp = df[(df['days_employed'].isnull()) & (df[column] == i)]
        temp1 = temp[column].count()
        res_temp1.append(temp1)
        result = (temp1 / counts)
       
        print(i, ': Количество пропущенных значений: ', temp1, sep='', end=' \n')
        res_percent = res_temp1[0] / counts[0]
        print('В процентах {:.0%}'.format(res_percent))
        del res_temp1[0]
        del counts[0]

Посмотрим на количество пропущенных значений для типов занятости:

In [12]:
skip_value('income_type')

сотрудник: Количество пропущенных значений: 1105 
В процентах 10%
компаньон: Количество пропущенных значений: 508 
В процентах 10%
пенсионер: Количество пропущенных значений: 413 
В процентах 11%
госслужащий: Количество пропущенных значений: 147 
В процентах 10%
безработный: Количество пропущенных значений: 0 
В процентах 0%
предприниматель: Количество пропущенных значений: 1 
В процентах 50%
студент: Количество пропущенных значений: 0 
В процентах 0%
в декрете: Количество пропущенных значений: 0 
В процентах 0%


Таким же образом оценим количество уникальных значений, а затем и пропуски в абсолютном и процентном соотношении для столбцов с: Количеством детей клиентов

In [13]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [14]:
skip_value('children')

0: Количество пропущенных значений: 1439 
В процентах 10%
1: Количество пропущенных значений: 475 
В процентах 10%
2: Количество пропущенных значений: 204 
В процентах 10%
3: Количество пропущенных значений: 36 
В процентах 11%
20: Количество пропущенных значений: 9 
В процентах 12%
-1: Количество пропущенных значений: 3 
В процентах 6%
4: Количество пропущенных значений: 7 
В процентах 17%
5: Количество пропущенных значений: 1 
В процентах 11%


Для возраста клиента:

In [15]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [16]:
skip_value('dob_years')

35: Количество пропущенных значений: 64 
В процентах 10%
40: Количество пропущенных значений: 66 
В процентах 11%
41: Количество пропущенных значений: 59 
В процентах 10%
34: Количество пропущенных значений: 69 
В процентах 11%
38: Количество пропущенных значений: 54 
В процентах 9%
42: Количество пропущенных значений: 65 
В процентах 11%
33: Количество пропущенных значений: 51 
В процентах 9%
39: Количество пропущенных значений: 51 
В процентах 9%
31: Количество пропущенных значений: 65 
В процентах 12%
36: Количество пропущенных значений: 63 
В процентах 11%
44: Количество пропущенных значений: 44 
В процентах 8%
29: Количество пропущенных значений: 50 
В процентах 9%
30: Количество пропущенных значений: 58 
В процентах 11%
48: Количество пропущенных значений: 46 
В процентах 9%
37: Количество пропущенных значений: 53 
В процентах 10%
50: Количество пропущенных значений: 51 
В процентах 10%
43: Количество пропущенных значений: 50 
В процентах 10%
32: Количество пропущенных значений: 

Для уровня образования клиентов

In [17]:
df['education'].str.lower().value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [18]:
skip_value('education')

среднее: Количество пропущенных значений: 1408 
В процентах 10%
высшее: Количество пропущенных значений: 496 
В процентах 11%
СРЕДНЕЕ: Количество пропущенных значений: 67 
В процентах 9%
Среднее: Количество пропущенных значений: 65 
В процентах 9%
неоконченное высшее: Количество пропущенных значений: 55 
В процентах 8%
ВЫСШЕЕ: Количество пропущенных значений: 23 
В процентах 8%
Высшее: Количество пропущенных значений: 25 
В процентах 9%
начальное: Количество пропущенных значений: 19 
В процентах 8%
Неоконченное высшее: Количество пропущенных значений: 7 
В процентах 15%
НЕОКОНЧЕННОЕ ВЫСШЕЕ: Количество пропущенных значений: 7 
В процентах 24%
НАЧАЛЬНОЕ: Количество пропущенных значений: 1 
В процентах 6%
Начальное: Количество пропущенных значений: 1 
В процентах 7%
ученая степень: Количество пропущенных значений: 0 
В процентах 0%
Ученая степень: Количество пропущенных значений: 0 
В процентах 0%
УЧЕНАЯ СТЕПЕНЬ: Количество пропущенных значений: 0 
В процентах 0%


Для семейного положения:

In [19]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [20]:
skip_value('family_status')

женат / замужем: Количество пропущенных значений: 1237 
В процентах 10%
гражданский брак: Количество пропущенных значений: 442 
В процентах 11%
Не женат / не замужем: Количество пропущенных значений: 288 
В процентах 10%
в разводе: Количество пропущенных значений: 112 
В процентах 9%
вдовец / вдова: Количество пропущенных значений: 95 
В процентах 10%


Для половой принадлежности:

In [21]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [22]:
skip_value('gender')

F: Количество пропущенных значений: 1484 
В процентах 10%
M: Количество пропущенных значений: 690 
В процентах 9%
XNA: Количество пропущенных значений: 0 
В процентах 0%


Для задолженности по возврату кредита

In [23]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [24]:
skip_value('debt')

0: Количество пропущенных значений: 2004 
В процентах 10%
1: Количество пропущенных значений: 170 
В процентах 10%


Для цели получения кредита

In [25]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [26]:
skip_value('purpose')

свадьба: Количество пропущенных значений: 76 
В процентах 10%
на проведение свадьбы: Количество пропущенных значений: 92 
В процентах 12%
сыграть свадьбу: Количество пропущенных значений: 81 
В процентах 10%
операции с недвижимостью: Количество пропущенных значений: 61 
В процентах 9%
покупка коммерческой недвижимости: Количество пропущенных значений: 67 
В процентах 10%
покупка жилья для сдачи: Количество пропущенных значений: 65 
В процентах 10%
операции с жильем: Количество пропущенных значений: 74 
В процентах 11%
операции с коммерческой недвижимостью: Количество пропущенных значений: 70 
В процентах 11%
жилье: Количество пропущенных значений: 60 
В процентах 9%
покупка жилья: Количество пропущенных значений: 52 
В процентах 8%
покупка жилья для семьи: Количество пропущенных значений: 71 
В процентах 11%
строительство собственной недвижимости: Количество пропущенных значений: 75 
В процентах 12%
недвижимость: Количество пропущенных значений: 62 
В процентах 10%
операции со своей не

**Вывод**

Как можно увидеть из приведённых выше данных, распределение пропусков для разных столбцов (исходя из процентного соотношения) примерно одинаково. Гипотеза о влиянии особенностей какой-либо категории клиентов на возникновение пропусков значений оказалась несостоятельной. Скорее всего речь может идти о ошибках загрузки информации, некорректном заполнении полей клиентами и пр. В силу этих обстоятельств, при заполнении пропущенных значений влияние категорий (столбцов) в расчёт браться не будут. В столбце с ежемесячным доходом наиболее адекватным способом заполнения пропусков видится медианное значение по типу занятости. Реализуем эту идею.

## Шаг 2. Предобработка данных

### Обработка пропусков

Определим медианное значение дохода по типу занятости

In [27]:
median_value_for_type_income = df.groupby('income_type')['total_income'].median()
median_value_for_type_income

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Напишем функцию для замены пропусков значений медианным значением по типу занятости в столбце 'total_income'

In [28]:
def replacing_the_revenue_pass(category, value):

    df.loc[(df['income_type'] == category) & (df['total_income'].isnull()), 'total_income'] = value
    return df.info()

In [29]:
replacing_the_revenue_pass('безработный', 131339.751676)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [30]:
replacing_the_revenue_pass('в декрете', 53829.130729)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [31]:
replacing_the_revenue_pass('госслужащий', 150447.935283)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19498 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [32]:
replacing_the_revenue_pass('компаньон', 172357.950966)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      20006 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [33]:
replacing_the_revenue_pass('пенсионер', 118514.486412)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      20419 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [34]:
replacing_the_revenue_pass('предприниматель', 499163.144947)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      20420 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [35]:
replacing_the_revenue_pass('сотрудник', 142594.396847)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [36]:
replacing_the_revenue_pass('студент', 98201.625314)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Проверим пропущенные значения в столбце 'total_income'

In [37]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Пропущенные значения в столбце ежемесячного дохода заполнены согласно медианным значениям выделенным по типу занятости

В представленной таблице есть ещё два столбца с проблематичными данными. В столбце 'children' присутствуют отрицательные значения, а также значения детей в количестве 20. А в столбце 'days_employed' пропущенные и отрицательные значения. Начнём со столбца "Дети" и выясним для начала, каков возраст родителей, имеющих по 20 детей.

In [38]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [39]:
df[(df['children'] == 20) & (df['dob_years'])]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
3302,20,NaN,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,150447.935283,профильное образование
3671,20,-913.161503,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля
3877,20,-8190.644409,45,среднее,1,гражданский брак,1,F,сотрудник,0,312270.145176,сделка с подержанным автомобилем
5020,20,-231.783475,37,среднее,1,гражданский брак,1,M,компаньон,0,260203.344629,сыграть свадьбу
5362,20,355898.021316,69,среднее,1,женат / замужем,0,M,пенсионер,0,138579.082863,автомобили
6198,20,NaN,35,высшее,0,Не женат / не замужем,4,M,компаньон,0,172357.950966,жилье
7516,20,-893.157059,45,среднее,1,женат / замужем,0,F,сотрудник,0,113956.567719,покупка жилья для сдачи
7789,20,360485.069162,55,среднее,1,женат / замужем,0,F,пенсионер,0,138626.756600,строительство собственной недвижимости


Как и ожидалось, это оказалось банальной ошибкой, поскольку вряд ли в 21, 23 и т.д. лет можно быть родителем 20 детей. Исправим это.

In [40]:
df['children'] = df['children'].replace(20, 2)

In [41]:
df['children'].value_counts()

 0    14149
 1     4818
 2     2131
 3      330
-1       47
 4       41
 5        9
Name: children, dtype: int64

Также заменим отрицательное значение

In [42]:
df['children'] = df['children'].replace(-1, 1)

In [43]:
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Чтобы составить поверхностное представление о столбце с трудовым стажем, посмотрим на его основные статистические данные.

In [44]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [45]:
401755 / 365

1100.6986301369864

Первое, что бросается в глаза, за исключением пропусков и отрицательных значений - максимальные выбросы с нереалистично завышенным количеством рабочих дней. ведь если взять максимальное значение и поделить на 365, то выяснится, что кто-то потратил на приобретение трудового стажа !1100! лет! Или, может быть, кто-то напутал и это не количество дней, а количество часов?

In [46]:
1100 / 24

45.833333333333336

45 лет гораздо более реалистичный срок трудового стажа, чем 1100 лет

Посмотрим как пересекаются данные из столбца с трудовым стажем, с такими как возраст и тип занятости

In [47]:
df[(df['days_employed']>0) & (df['dob_years'])]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили
...,...,...,...,...,...,...,...,...,...,...,...,...
21501,0,334343.096304,57,среднее,1,женат / замужем,0,F,пенсионер,0,86232.126549,жилье
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


In [48]:
df[(df['days_employed']>0) & (df['dob_years'])].count()

children            1693
days_employed       1693
dob_years           1693
education           1693
education_id        1693
family_status       1693
family_status_id    1693
gender              1693
income_type         1693
debt                1693
total_income        1693
purpose             1693
dtype: int64

In [49]:
df[(df['days_employed']>0) & (df['income_type']=='пенсионер')].count()

children            3443
days_employed       3443
dob_years           3443
education           3443
education_id        3443
family_status       3443
family_status_id    3443
gender              3443
income_type         3443
debt                3443
total_income        3443
purpose             3443
dtype: int64

In [50]:
df[df['days_employed']>0].count()

children            3445
days_employed       3445
dob_years           3445
education           3445
education_id        3445
family_status       3445
family_status_id    3445
gender              3445
income_type         3445
debt                3445
total_income        3445
purpose             3445
dtype: int64

Получается, что положительные трудовые дни, практически все (за исключение 2-х) принадлежат к типу занятости - пенсионеры. Может быть потому, что их трудовой стаж уже завершён, положен на полочку и, так сказать, никуда не движется? Посмотрим, а есть ли в положительных значениях неоправданно большие выбросы?

In [51]:
df[df['days_employed']>0].max()

children                          4
days_employed                401755
dob_years                        74
education            ученая степень
education_id                      4
family_status       женат / замужем
family_status_id                  4
gender                            M
income_type               пенсионер
debt                              1
total_income                 735103
purpose             сыграть свадьбу
dtype: object

In [52]:
df[df['days_employed']>0].min()

children                                0
days_employed                      328729
dob_years                               0
education                          ВЫСШЕЕ
education_id                            0
family_status       Не женат / не замужем
family_status_id                        0
gender                                  F
income_type                   безработный
debt                                    0
total_income                      20667.3
purpose                        автомобили
dtype: object

In [53]:
328729 / 24

13697.041666666666

Получается, что все положительные значения не только принадлежат категории "пенсионеры", но ещё и указаны в часах.

Рассмотрим отрицательные значения

In [54]:
df[(df['days_employed']< 0) & (df['dob_years'])]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи
...,...,...,...,...,...,...,...,...,...,...,...,...
21511,0,-612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21513,0,-1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,-280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости


In [55]:
df[df['days_employed']>0].min()

children                                0
days_employed                      328729
dob_years                               0
education                          ВЫСШЕЕ
education_id                            0
family_status       Не женат / не замужем
family_status_id                        0
gender                                  F
income_type                   безработный
debt                                    0
total_income                      20667.3
purpose                        автомобили
dtype: object

Заменим пока, для удобства работы, пропуски в трудовом стаже нулями, приведем к годовому счислению, заменим тип данных на int и выделим в отдельный столбец "years_employed"

In [56]:
df['days_employed'] = df['days_employed'].fillna(0)

In [57]:
df['years_employed'] = df['days_employed'] / 365

In [58]:
df['years_employed'] = df['years_employed'].astype(int).abs()

In [59]:
df.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,2
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,7
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,0
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,18
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,5


**Вывод**

### Замена типа данных

In [60]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
years_employed      0
dtype: int64

In [61]:
temp = df[df['years_employed'] < 50]['years_employed'].mean()

In [62]:
temp

5.228331213009569

Среднее значение среди данных по годам рабочего стажа (кроме тех, что с завышенным значением) равно 5 лет. Заменим этим числом все нулевые значения.

In [63]:
df.loc[df['years_employed'] == 0, 'years_employed']=5

In [64]:
display(df.head(30))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,2
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,7
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,5
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,18
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,5


Приведём значения в столбцу с образованием к нижнему регистру. Посмотрим, как изменились уникальные значения по сравнению с прошлым разом

In [65]:
df['education'].str.lower().value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Ещё раз пробежимся по таблице в поисках строк в верхнем регистре, уникальных значений, неверных типов данных и артефактов

In [66]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose', 'years_employed'],
      dtype='object')

In [67]:
df['education'].str.lower().value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [68]:
df['family_status'].str.lower().value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [69]:
df['gender'].str.lower().value_counts()

f      14236
m       7288
xna        1
Name: gender, dtype: int64

В столбце половой принадлежности встречаем непонятное значение 'xna'. Что за зверь такой? Поисковик подсказывает, что существует фреймворк для игровых разработчиков с таким именем - вряд ли это имеет отношение к половой принадлежности. Возможно аббревиатура, ещё возможнее - неверный ввод (если предположить, что ошиблись раскладкой клавиатуры - получится "чтф" - тоже малоинформативное значение). В итоге принято решение оставить 'xna' без изменений, особенно учитывая его единичное значение.

Заменим тип данных в столбце "total_income" на int.

In [70]:
df['total_income'] = df['total_income'].astype(int)

In [71]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932


Обращает на себя внимание явная ошибка с доходами - предположим, что по отчётности сместили запятую на один знак влево. Проверим это предположение.

In [72]:
df.groupby('income_type')['total_income'].mean()

income_type
безработный        131339.000000
в декрете           53829.000000
госслужащий        168837.299520
компаньон          199413.927237
пенсионер          135133.413641
предприниматель    499163.000000
сотрудник          159512.838205
студент             98201.000000
Name: total_income, dtype: float64

In [73]:
df.groupby('income_type')['total_income'].median()

income_type
безработный        131339
в декрете           53829
госслужащий        150447
компаньон          172357
пенсионер          118514
предприниматель    499163
сотрудник          142594
студент             98201
Name: total_income, dtype: int32

Предположение приобретает удивительно правдоподобные очертания: студент со средним ежемесячным доходом в 98 тысяч рублей, пенсионер с медианной пенсией в 118 тысяч рублей... Впрочем, не будучи знакомым с особенностями выборки данного банка (может именно такая выборка и является корректной), оставим всё как есть. Тем более, что равномерное, без выбросов, увеличение дохода в десять раз для всех не отразится некорректно на окончательных результатах.

Взглянем ещё раз на цели получения кредита

In [74]:
df['purpose'].str.lower().value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Пора обработать дубли и приступать к лемматизации

**Вывод**

### Обработка дубликатов

Теперь изучим дубли

In [75]:
df = df.drop_duplicates().reset_index(drop=True)

In [76]:
# Проверим, остались ли дубли. 
df.duplicated().sum()

0

**Вывод**

Обнаружены явные дубли в размере 54 штук. Дубли успешно удалены.

### Лемматизация

Лемматизируем данные столбца 'purpose'. Для этого присвоим переменной количество уникальных значений столбца и преобразуем в список методом tolist()

In [77]:
unic_purpose = df['purpose'].unique().tolist()
unic_purpose

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

Теперь меитодом join преобразуем список в строку и выведем на экран

In [78]:
text = ' '.join(unic_purpose)
text

'покупка жилья приобретение автомобиля дополнительное образование сыграть свадьбу операции с жильем образование на проведение свадьбы покупка жилья для семьи покупка недвижимости покупка коммерческой недвижимости покупка жилой недвижимости строительство собственной недвижимости недвижимость строительство недвижимости на покупку подержанного автомобиля на покупку своего автомобиля операции с коммерческой недвижимостью строительство жилой недвижимости жилье операции со своей недвижимостью автомобили заняться образованием сделка с подержанным автомобилем получение образования автомобиль свадьба получение дополнительного образования покупка своего жилья операции с недвижимостью получение высшего образования свой автомобиль сделка с автомобилем профильное образование высшее образование покупка жилья для сдачи на покупку автомобиля ремонт жилью заняться высшим образованием'

Лемматизируем содержание переменной text

In [79]:
lemmas = m.lemmatize(text)
lemmas

['покупка',
 ' ',
 'жилье',
 ' ',
 'приобретение',
 ' ',
 'автомобиль',
 ' ',
 'дополнительный',
 ' ',
 'образование',
 ' ',
 'сыграть',
 ' ',
 'свадьба',
 ' ',
 'операция',
 ' ',
 'с',
 ' ',
 'жилье',
 ' ',
 'образование',
 ' ',
 'на',
 ' ',
 'проведение',
 ' ',
 'свадьба',
 ' ',
 'покупка',
 ' ',
 'жилье',
 ' ',
 'для',
 ' ',
 'семья',
 ' ',
 'покупка',
 ' ',
 'недвижимость',
 ' ',
 'покупка',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ' ',
 'покупка',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'собственный',
 ' ',
 'недвижимость',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'недвижимость',
 ' ',
 'на',
 ' ',
 'покупка',
 ' ',
 'подержать',
 ' ',
 'автомобиль',
 ' ',
 'на',
 ' ',
 'покупка',
 ' ',
 'свой',
 ' ',
 'автомобиль',
 ' ',
 'операция',
 ' ',
 'с',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 ' ',
 'жилье',
 ' ',
 'операция',
 ' ',
 'со',
 ' ',
 'свой',
 ' ',
 'недвижимо

Подсчитаем количество упоминаний в тексте

In [80]:
from collections import Counter
print(Counter(lemmas)) 

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


**Вывод**

После окончания процесса лемматизации, мы получили словарь с ключами и значениями целей для взятия кредита. В первую четвёрку по причинам взятия кредита попали:
- 'недвижимость'
- 'автомобиль'
- 'образование'
- 'жилье'
Потом, в порядке снижения приоритета:
- 'свадьба'
- 'строительство'
- 'ремонт'

### Категоризация данных

В процессе отетов на вопросы банка, мы сравним данные между собой взяв их из интересующих бизнес категорий. Для начала сопоставим наличие детей и возврат/невозврат кредита.

# Зависимость между наличием детей и возвратом кредита в срок

Создадим таблицу из имеющейся со столбцами 'debt' и 'children'

In [81]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose', 'years_employed'],
      dtype='object')

In [82]:
influence_child_debt = df[['debt', 'children']]
influence_child_debt.head(15)

,debt,children
0,0,1
1,0,1
2,0,0
3,0,3
4,0,0
5,0,0
6,0,0
7,0,0
8,0,2
9,0,0


Поищем среднее значение во взаимосвязи детей и возврата кредита

In [83]:
influence_child_debt.groupby('debt').mean().sort_values('children',ascending=False).head(10)

,children
debt,
1,0.543366
0,0.474658


In [84]:
influence_child_debt.groupby('children').mean().sort_values('debt',ascending=False).head(10)

,debt
children,
4,0.097561
2,0.094925
1,0.091639
3,0.081818
0,0.075353
5,0.000000


In [85]:
influence_child_debt[influence_child_debt['children']==5]['children'].count()

9

In [86]:
pivot_child_debt = df.pivot_table(index = ['children'],
columns = 'debt', 
values = 'family_status', aggfunc='count')

pivot_child_debt['result'] = pivot_child_debt[1] / pivot_child_debt[0] * 100
pivot_child_debt

debt,0,1,result
children,,,
0,13044.0,1063.0,8.149341
1,4411.0,445.0,10.088415
2,1926.0,202.0,10.488058
3,303.0,27.0,8.910891
4,37.0,4.0,10.810811
5,9.0,NaN,NaN


**Вывод**

Как видим из таблицы, разница между максимальными и минимальными показателями составляет 2 процента. Самый низкий процент невозврата (иначе говоря самые добросовестные клиенты по выплате) кредита показывают бездетные и родители троих детей. Существенного воздействия наличие и количество детей на возврат кредита не оказывает.

# Зависимость между семейным положением и возвратом кредита в срок

In [87]:
influence_family_debt = df[['debt', 'family_status']]
influence_family_debt.head(15)

,debt,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,0,гражданский брак
5,0,гражданский брак
6,0,женат / замужем
7,0,женат / замужем
8,0,гражданский брак
9,0,женат / замужем


In [88]:
influence_family_debt.groupby('family_status')['debt'].mean()

family_status
Не женат / не замужем    0.097509
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093202
женат / замужем          0.075421
Name: debt, dtype: float64

In [89]:
pivot_family_debt = df.pivot_table(index = ['family_status'],
columns = 'debt', 
values = 'children', aggfunc='count')

pivot_family_debt['result'] = pivot_family_debt[1] / pivot_family_debt[0] * 100
pivot_family_debt

debt,0,1,result
family_status,,,
Не женат / не замужем,2536,274,10.804416
в разводе,1110,85,7.657658
вдовец / вдова,896,63,7.031250
гражданский брак,3775,388,10.278146
женат / замужем,11413,931,8.157364


# Вывод

Разбежность показателей между максимальными и минимальными значениями составляет уже чуть меньше четырёх процентов, что почти вдвое превышает сравнение по детям. Напрашивается вывод, что семейный статус более значимый критерий для рассмотрения при выдаче кредитов. Как ни странно, самыми лучшими плательщиками, исходя из полученных процентов, являются разведённые и вдовцы. Впрочем, эти категории, в то же время, являются и самыми малочисленными (каждая из этих категорий, например, представлена в количестве на порядок меньше, чем женатые). Напротив, клиенты не женатые и состоящие в гражданском браке оказались самыми недобросовестными плательщиками. Впрочем, ещё раз оговоримся, разница в семейном положении всё же не может считаться ни решающей, ни значительной.

# Зависимость между уровнем дохода и возвратом кредита в срок

In [90]:
influence_income_debt = df[['debt', 'total_income']]
influence_income_debt.head(15)

,debt,total_income
0,0,253875
1,0,112080
2,0,145885
3,0,267628
4,0,158616
5,0,255763
6,0,240525
7,0,135823
8,0,95856
9,0,144425


In [91]:
influence_income_debt.groupby('total_income')['debt'].mean()

total_income
20667      1.0
21205      0.0
21367      0.0
21695      0.0
21895      0.0
          ... 
1711309    0.0
1715018    0.0
1726276    0.0
2200852    1.0
2265604    0.0
Name: debt, Length: 18608, dtype: float64

In [92]:
influence_income_debt['total_income'].max()

2265604

In [93]:
influence_income_debt['total_income'].min()

20667

Отличие на два порядка между максимальным и минимальным уровнем дохода. Тем Интереснее будет посмотреть на результаты.

In [94]:
influence_income_debt['total_income'].mean()

165295.3792557403

In [95]:
influence_income_debt['total_income'].median()

142594.0

In [96]:
influence_income_debt[influence_income_debt['total_income'] > 1000000]['total_income'].count()

25

In [97]:
influence_income_debt[influence_income_debt['total_income'] < 100000]['total_income'].count()

4463

Напишем функцию, которая поможет нам соотнести уровень дохода и возвращение кредита.

In [98]:
def income_value(total_income):
    if total_income <= 100000:
        return 'низкий доход'
    if 100000 < total_income <= 500000:
        return 'средний доход'
    if 500000 < total_income <= 1000000:
        return 'высокий доход'
    if total_income > 1000000:
        return 'очень высокий доход'

In [99]:
df['income_value'] = df['total_income'].apply(income_value)

In [100]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,income_value
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,средний доход
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,средний доход
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,средний доход
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,средний доход
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932,средний доход
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,12,средний доход
21467,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,942,средний доход
21468,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,5,низкий доход
21469,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,8,средний доход


In [101]:
pivot_income_debt = df.pivot_table(index = ['income_value'],
columns = 'debt', 
values = 'purpose', aggfunc='count')

pivot_income_debt['result'] = pivot_income_debt[1] / pivot_income_debt[0] * 100
pivot_income_debt

debt,0,1,result
income_value,,,
высокий доход,185,12,6.486486
низкий доход,4109,354,8.615235
очень высокий доход,23,2,8.695652
средний доход,15413,1373,8.908065


# Выводы

Выводы и на сей раз не смогут порадовать нас своей категоричностью и однозначностью. Значения доходов приравненные к низким, средним и очень высоким практически идентичны по своим процентным соотношениям. Выделяется только категория с высоким доходом: представители этой категории примерно на 2 процента с большей вероятноятностью вернут кредиты по сравнению с другими. Впрочем, как и в других сравнениях по столбцам, принципиальных отличий опять выявлено не было. 

# Как разные цели кредита влияют на его возврат в срок

In [102]:
influence_purpose_debt = df[['debt', 'purpose']]
influence_purpose_debt.head(15)
           
            

,debt,purpose
0,0,покупка жилья
1,0,приобретение автомобиля
2,0,покупка жилья
3,0,дополнительное образование
4,0,сыграть свадьбу
5,0,покупка жилья
6,0,операции с жильем
7,0,образование
8,0,на проведение свадьбы
9,0,покупка жилья для семьи


In [103]:
def lemmas_for_debt(column):
    
    if 'свадь' in column:
        return 'свадьба'
    elif 'авто' in column:
        return 'автомобиль'
    elif 'образов' in column:            
        return 'образование'
    else:
        return 'жилье'  
        

In [104]:
df['clean_purpose'] = df['purpose'].apply(lemmas_for_debt)

In [105]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,income_value,clean_purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,средний доход,жилье
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,средний доход,автомобиль
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,средний доход,жилье
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,средний доход,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932,средний доход,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,12,средний доход,жилье
21467,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,942,средний доход,автомобиль
21468,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,5,низкий доход,жилье
21469,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,8,средний доход,автомобиль


In [106]:
df['clean_purpose'].value_counts()

жилье          10814
автомобиль      4308
образование     4014
свадьба         2335
Name: clean_purpose, dtype: int64

In [107]:
influence_purpose_debt = df[['debt', 'clean_purpose']]
influence_purpose_debt.head(15)

,debt,clean_purpose
0,0,жилье
1,0,автомобиль
2,0,жилье
3,0,образование
4,0,свадьба
5,0,жилье
6,0,жилье
7,0,образование
8,0,свадьба
9,0,жилье


In [108]:
pivot_purpose_debt = df.pivot_table(index = ['clean_purpose'],
columns = 'debt', 
values = 'income_value', aggfunc='count')

pivot_purpose_debt['result'] = pivot_purpose_debt[1] / pivot_purpose_debt[0] * 100
pivot_purpose_debt

debt,0,1,result
clean_purpose,,,
автомобиль,3905,403,10.320102
жилье,10032,782,7.795056
образование,3644,370,10.153677
свадьба,2149,186,8.655188


# Выводы

Самой частой целью взятия кредита в абсолютных значениях (в среднем в 3 раза чаще, чем для любого другого обращения) - является приобретение жилья. Эта же категория является лидером по возврату кредита - примерно на 2,5 процента больше возвратов, чем на приобретение автомобиля. На втором месте по добропорядочности в возврате кредитов стоят те, кто берёт кредит на свадьбу. Кредиты на автомобиль и образование возвращают с меньшей охотой.

## Шаг 3. Ответьте на вопросы

Перед контрольными ответами на вопросы хотелось бы ещё раз заметить, что кардинальных отличий в зависимостях разных категорий на возврат кредита выявлено не было.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

**Вывод**

Самыми ответственными плательщиками показали себя бездетные и родители троих детей

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

**Вывод**

Лучшие плательщики - разведённые и вдовцы

Худшие показатели - не женатых и людей, состоящих в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**Вывод**

Люди с высоким доходом на 2 процента чаще возвращают кредит в срок, чем люди с низким, средним и очень высоким доходами.

- Как разные цели кредита влияют на его возврат в срок?

**Вывод**

Жильё - по показателям - лучший способ кредитования с точки зрения возврата кредита. Кредиты на автомобиль и образование - возврат примерно на 2,5 процентов ниже. 

## Шаг 4. Общий вывод

Проведённое исследование показало некоторые незначительные колебания в изучаемых категориях. Для более убедительного определения значимости полученных результатов, необходимо применение методов математической статистики. 
Кроме описанных выше выводов, хотелось отметить ряд вполне достойных изучения и сравнения категорий:
по половой принадлежности, образованию...Также было бы полезно и интересно узнать репрезентативность выборки - насколько представленное распределение ха рактерно для генеральной совокупности, есть ли отличия и если да, то чем они могли бы быть обусловлены. Кажется неочевидные выводы могли бы вполне удивить и быть полезны. 

В таблице осталось немало мусора, но поскольку этот мусор никак не влиял на проведение анализа, его решено было оставить либо в неизменности, либо в небольшой модификации - достаточной для получения той или иной информации.

Пропуски, обнаруженные в таблице, при детальном исследовании не принесли никакой новой информации о возможной причине своего возникновения.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.